In [1]:
from selenium.common.exceptions import TimeoutException
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from appium.options.android import UiAutomator2Options
from appium import webdriver
import pandas as pd
from datetime import datetime
import time

# ===================== 설정 =====================
APP_PACKAGE = "kr.or.keis.mo"

LOGIN_ID = "aaaa5380"
LOGIN_PW = "abcd1234!"

APPIUM_SERVER_URL = "http://127.0.0.1:4723"
DEVICE_NAME = "Android"


# ===================== 공통 동작 함수 =====================
def tap_idpw_menu(driver, wait):
    """'아이디/비밀번호(HRD 출결용)' 버튼 클릭"""
    print("📲 [3단계] '아이디/비밀번호(HRD 출결용)' 버튼 클릭")

    # 화면 아래쪽 보이도록 한 번 스와이프
    try:
        driver.swipe(22, 942, 22, 650, 500)  # (startX, startY, endX, endY)
        time.sleep(1)
        print("   ✅ 스와이프 수행 완료")
    except Exception as e:
        print(f"   ℹ️ 스와이프 중 오류(무시): {e}")

    # UiSelector(resourceId) 로 버튼 클릭
    try:
        btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ANDROID_UIAUTOMATOR,
                 'new UiSelector().resourceId("btn_idpopup")')
            )
        )
        btn.click()
        print("   ✅ '아이디/비밀번호(HRD 출결용)' 버튼 클릭 성공")
    except TimeoutException as e:
        print("   ❌ '아이디/비밀번호(HRD 출결용)' 버튼을 찾지 못했습니다.")
        raise RuntimeError("ID/PW(HRD 출결용) 메뉴를 찾지 못했습니다.") from e


def fill_login_form(driver, wait):
    """로그인 화면에서 ID/PW 입력"""
    print("⌨️ [4단계] 아이디 / 비밀번호 자동 입력")

    def _two_edittexts(d):
        els = d.find_elements(AppiumBy.CLASS_NAME, "android.widget.EditText")
        return els if len(els) >= 2 else False

    try:
        inputs = wait.until(_two_edittexts)
        id_input, pw_input = inputs[0], inputs[1]

        id_input.clear()
        id_input.send_keys(LOGIN_ID)
        print("   ✅ ID 입력 완료")

        pw_input.clear()
        pw_input.send_keys(LOGIN_PW)
        print("   ✅ PW 입력 완료")
    except Exception as e:
        print("   ❌ 로그인 입력창을 찾지 못했습니다.")
        raise RuntimeError("로그인 ID/PW 입력창을 찾지 못했습니다.") from e


# ===================== 메인 테스트 =====================
def test_login_security_safe():

    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = ".MainActivity"
    options.automation_name = "UiAutomator2"
    options.new_command_timeout = 300
    options.no_reset = True

    print("--- 로그인 성능 테스트 ---")
    driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
    wait = WebDriverWait(driver, 20)

    elapsed = None
    popup_first_line = ""

    try:
        # [1단계] 전체메뉴 버튼
        print("📲 [1단계] 메인 -> 전체메뉴 버튼 클릭")
        all_menu_btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.XPATH, "//android.widget.Button[@text='전체메뉴']")
            )
        )
        all_menu_btn.click()

        # [2단계] '로그인을 해 주세요'
        print("📲 [2단계] '로그인을 해 주세요' 영역 클릭")
        login_please = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ACCESSIBILITY_ID, "로그인을 해 주세요")
            )
        )
        login_please.click()

        print("⏳ [2-1단계] 화면 전환 안정화를 위해 3초 대기...")
        time.sleep(3)

        # [3단계] 아이디/비밀번호(HRD 출결용)
        tap_idpw_menu(driver, wait)

        # [4단계] ID/PW 입력
        fill_login_form(driver, wait)

        # [5단계] 로그인 버튼 클릭 + 응답 시간 측정
        print("⏱️ [5단계] 로그인 버튼 클릭 후 응답 속도 측정 시작")

        # 키보드 숨기기 (가려져 있을 수 있으니)
        try:
            driver.hide_keyboard()
            time.sleep(0.5)
            print("   📱 소프트 키보드 숨김")
        except Exception:
            pass

        # ✅ 여기까지는 준비 단계, 아직 시간 재지 않음 (버튼 찾기까지 포함 X)
        login_btn = wait.until(
            EC.element_to_be_clickable(
                (AppiumBy.ANDROID_UIAUTOMATOR,
                 'new UiSelector().resourceId("btnIndvIdLogin")')
            )
        )

        # ✅ 시간을 재는 구간: 버튼을 누르는 순간 ~ 팝업이 나타날 때까지
        start_time = time.time()
        login_btn.click()

        popup = wait.until(
            EC.presence_of_element_located((AppiumBy.ID, "android:id/message"))
        )
        end_time = time.time()

        elapsed = end_time - start_time
        popup_first_line = popup.text.splitlines()[0] if popup.text else ""

        print("\n🎉 로그인 응답 수신!")
        print(f"🚀 로그인 반응 속도: {elapsed:.4f} 초")
        print(f"📄 팝업 내용: {popup_first_line}")

        # 팝업 확인 버튼
        try:
            driver.find_element(AppiumBy.ID, "android:id/button1").click()
        except Exception:
            pass

        print("✅ 테스트 완료")

    except Exception as e:
        print("\n❌ 테스트 실패")
        print(f"에러 내용: {e}")

    finally:
        driver.quit()
        print("\n✅ 드라이버 종료 완료")

    # ===================== CSV 저장 =====================
    print("\n" + "=" * 50)
    print("💾 로그인 성능 결과 CSV 저장 중...")

    if elapsed is not None:
        df = pd.DataFrame([{
            "측정시각": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "로그인반응속도(초)": round(elapsed, 4),
            "팝업메시지": popup_first_line,
        }])

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"login_perf_result_{timestamp}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")

        print(f"✅ CSV 저장 완료! 파일명: {file_name}")
        print(df)
    else:
        print("ℹ️ 측정된 시간이 없어 CSV는 생성하지 않습니다.")

    print("\n✅ 로그인 테스트 및 CSV 저장까지 모두 완료되었습니다.")


if __name__ == "__main__":
    test_login_security_safe()


--- 로그인 성능 테스트 ---
📲 [1단계] 메인 -> 전체메뉴 버튼 클릭
📲 [2단계] '로그인을 해 주세요' 영역 클릭
⏳ [2-1단계] 화면 전환 안정화를 위해 3초 대기...
📲 [3단계] '아이디/비밀번호(HRD 출결용)' 버튼 클릭
   ✅ 스와이프 수행 완료
   ✅ '아이디/비밀번호(HRD 출결용)' 버튼 클릭 성공
⌨️ [4단계] 아이디 / 비밀번호 자동 입력
   ✅ ID 입력 완료
   ✅ PW 입력 완료
⏱️ [5단계] 로그인 버튼 클릭 후 응답 속도 측정 시작
   📱 소프트 키보드 숨김

🎉 로그인 응답 수신!
🚀 로그인 반응 속도: 2.3804 초
📄 팝업 내용: 로그인 성공하였습니다.
✅ 테스트 완료

✅ 드라이버 종료 완료

💾 로그인 성능 결과 CSV 저장 중...
✅ CSV 저장 완료! 파일명: login_perf_result_20251203_160736.csv
                  측정시각  로그인반응속도(초)         팝업메시지
0  2025-12-03 16:07:36      2.3804  로그인 성공하였습니다.

✅ 로그인 테스트 및 CSV 저장까지 모두 완료되었습니다.
